## PCI CONFLICT DETECTION AND AUTOMATIC PCI PLANNING (APP)

#### Importing required libraries

In [1]:
# Import the required libraries
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#### Read in the csv file into a dataframe

In [2]:
df = pd.read_csv("D:\AA_Download\CSV_4G_PCI_Matje.csv") # read in the csv file

#### Explore the dataframe and clean the data

In [3]:
df # check file contents and rows/columns

,Cell name,LONG,LAT,PCI,Territory
0,L0002A,3.355111,6.45301,273,APAPA
1,L0002B,3.355111,6.45301,274,APAPA
2,L0002C,3.355111,6.45301,275,APAPA
3,L0002D,3.355111,6.45301,273,APAPA
4,L0002E,3.355111,6.45301,274,APAPA
...,...,...,...,...,...
41330,LOG4929J,3.724694,6.896167,447,IJEBU-ODE
41331,LOG5543K,3.7176,6.89383,348,IJEBU-ODE
41332,LOG5543B,3.7176,6.89383,348,IJEBU-ODE
41333,LOG4929K,3.724694,6.896167,448,IJEBU-ODE


In [4]:
df.isna().sum() # missing entry check

Cell name    0
LONG         0
LAT          0
PCI          0
Territory    0
dtype: int64

In [5]:
df["Territory"].unique() # unique territory checks

array(['APAPA', 'LAGOS ISLAND', 'IKEJA', 'KOSOFE', 'ALIMOSHO',
       'LAGOS MAINLAND', 'SANGO', 'FESTAC', 'MUSHIN', 'AGEGE',
       'IJEBU-ODE', 'ABEOKUTA', 'IKORODU', 'OGBOMOSHO', 'Ikeja',
       'ETI OSA', 'BADAGRY', 'SURULERE', ' IJEBU-ODE', 'EPE', 'Ikorodu',
       'LAGOS', 'OGUN WATERSIDE', 'ONDO'], dtype=object)

In [6]:
df.duplicated().sum() # checking for duplicates

18

In [7]:
# Remove duplicates based on Cell name
df = df.drop_duplicates(subset='Cell name', keep='first').reset_index(drop=True)

In [8]:
df.duplicated().sum() # confirming duplicate removal

0

In [9]:
df.info() # checking datatype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41302 entries, 0 to 41301
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Cell name  41302 non-null  object
 1   LONG       41302 non-null  object
 2   LAT        41302 non-null  object
 3   PCI        41302 non-null  int64 
 4   Territory  41302 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.6+ MB


In [10]:
df["LONG"] = df["LONG"].astype(float) # converting longitude to float
df["LAT"] = df["LAT"].astype(float) # converting latitude to float

In [11]:
df.info() # cross-checking datatype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41302 entries, 0 to 41301
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Cell name  41302 non-null  object 
 1   LONG       41302 non-null  float64
 2   LAT        41302 non-null  float64
 3   PCI        41302 non-null  int64  
 4   Territory  41302 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 1.6+ MB


#### Automation: Find PCI conflicts within 1km and replan where necessary

In [12]:
# importing required library to compute geo-distance
from geopy.distance import geodesic

### IKEJA TERRITORY

In [13]:
# Filter DataFrame to only include cells in the "IKEJA" territory
df_ikeja = df[df['Territory'] == 'IKEJA']

# Function to calculate distance between all cells within a specified territory and replan clashed PCIs
def calculate_distances_and_replan_pci(df, distance_range=(0.1, 1), available_pcis=range(0, 504)):
    pci_clashes = []
    cell_names = df['Cell name'].values
    latitudes = df['LAT'].values
    longitudes = df['LONG'].values
    pcis = df['PCI'].values
    used_pcis = set(pcis)
    
    for i in range(len(df)):
        coord1 = (latitudes[i], longitudes[i])
        
        for j in range(i + 1, len(df)):
            coord2 = (latitudes[j], longitudes[j])
            distance = geodesic(coord1, coord2).kilometers
            
            # Check for PCI clash within the specified distance range
            if distance_range[0] <= distance < distance_range[1] and pcis[i] == pcis[j]:
                pci_clashes.append((cell_names[i], cell_names[j], pcis[i], distance))
                
                # Find new PCI for one of the clashing cells
                for new_pci in available_pcis:
                    if new_pci not in used_pcis:
                        df.loc[df['Cell name'] == cell_names[j], 'PCI'] = new_pci
                        used_pcis.add(new_pci)
                        break
    
    return pci_clashes

# Calculate distances and replan clashed PCIs in the "IKEJA" territory
pci_clashes = calculate_distances_and_replan_pci(df_ikeja)

# Display PCI clashes and the new DataFrame with updated PCI values
if pci_clashes:
    print("PCI Clashes found and replanned:")
    for clash in pci_clashes:
        print(f"Cells {clash[0]} and {clash[1]} had the same PCI ({clash[2]}) and were {clash[3]:.2f} km apart.")
    
    # Save the updated DataFrame to a CSV file
    df_ikeja.to_csv('updated_ikeja_cells.csv', index=False)
    print("\nUpdated DataFrame saved to 'updated_ikeja_cells.csv'.")
else:
    print("No PCI clashes found within the distance range of 0.1 km to 1 km.")

No PCI clashes found within the distance range of 0.1 km to 1 km.


### KOSOFE TERRITORY

In [15]:
# Filter DataFrame to only include cells in the "KOSOFE" territory
df_kosofe = df[df['Territory'] == 'KOSOFE']

# Function to calculate distance between all cells within a specified territory and replan clashed PCIs
def calculate_distances_and_replan_pci(df, distance_range=(0.1, 1), available_pcis=range(0, 504)):
    pci_clashes = []
    cell_names = df['Cell name'].values
    latitudes = df['LAT'].values
    longitudes = df['LONG'].values
    pcis = df['PCI'].values
    used_pcis = set(pcis)
    
    for i in range(len(df)):
        coord1 = (latitudes[i], longitudes[i])
        
        for j in range(i + 1, len(df)):
            coord2 = (latitudes[j], longitudes[j])
            distance = geodesic(coord1, coord2).kilometers
            
            # Check for PCI clash within the specified distance range
            if distance_range[0] <= distance < distance_range[1] and pcis[i] == pcis[j]:
                pci_clashes.append((cell_names[i], cell_names[j], pcis[i], distance))
                
                # Find new PCI for one of the clashing cells
                for new_pci in available_pcis:
                    if new_pci not in used_pcis:
                        df.loc[df['Cell name'] == cell_names[j], 'PCI'] = new_pci
                        used_pcis.add(new_pci)
                        break
    
    return pci_clashes

# Calculate distances and replan clashed PCIs in the "KOSOFE" territory
pci_clashes = calculate_distances_and_replan_pci(df_kosofe)

# Display PCI clashes and the new DataFrame with updated PCI values
if pci_clashes:
    print("PCI Clashes found and replanned:")
    for clash in pci_clashes:
        print(f"Cells {clash[0]} and {clash[1]} had the same PCI ({clash[2]}) and were {clash[3]:.2f} km apart.")
    
    # Save the updated DataFrame to a CSV file
    df_kosofe.to_csv('updated_kosofe_cells.csv', index=False)
    print("\nUpdated DataFrame saved to 'updated_kosofe_cells.csv'.")
else:
    print("No PCI clashes found within the distance range of 0.1 km to 1 km.")


PCI Clashes found and replanned:
Cells LLG3203A and LLG3621A had the same PCI (192) and were 0.40 km apart.
Cells LLG3203A and LLG3621D had the same PCI (192) and were 0.40 km apart.
Cells LLG3203A and LLG3621G had the same PCI (192) and were 0.40 km apart.
Cells LLG3203A and LLG3621X had the same PCI (192) and were 0.40 km apart.
Cells LLG3203B and LLG3621B had the same PCI (193) and were 0.40 km apart.
Cells LLG3203B and LLG3621E had the same PCI (193) and were 0.40 km apart.
Cells LLG3203B and LLG3621H had the same PCI (193) and were 0.40 km apart.
Cells LLG3203B and LLG3621Y had the same PCI (193) and were 0.40 km apart.
Cells LLG3203C and LLG3621C had the same PCI (194) and were 0.40 km apart.
Cells LLG3203C and LLG3621F had the same PCI (194) and were 0.40 km apart.
Cells LLG3203C and LLG3621I had the same PCI (194) and were 0.40 km apart.
Cells LLG3203C and LLG3621Z had the same PCI (194) and were 0.40 km apart.

Updated DataFrame saved to 'updated_kosofe_cells.csv'.


### AGEGE TERRITORY

In [16]:
# Filter DataFrame to only include cells in the "AGEGE" territory
df_agege = df[df['Territory'] == 'AGEGE']

# Function to calculate distance between all cells within a specified territory and replan clashed PCIs
def calculate_distances_and_replan_pci(df, distance_range=(0.1, 1), available_pcis=range(0, 504)):
    pci_clashes = []
    cell_names = df['Cell name'].values
    latitudes = df['LAT'].values
    longitudes = df['LONG'].values
    pcis = df['PCI'].values
    used_pcis = set(pcis)
    
    for i in range(len(df)):
        coord1 = (latitudes[i], longitudes[i])
        
        for j in range(i + 1, len(df)):
            coord2 = (latitudes[j], longitudes[j])
            distance = geodesic(coord1, coord2).kilometers
            
            # Check for PCI clash within the specified distance range
            if distance_range[0] <= distance < distance_range[1] and pcis[i] == pcis[j]:
                pci_clashes.append((cell_names[i], cell_names[j], pcis[i], distance))
                
                # Find new PCI for one of the clashing cells
                for new_pci in available_pcis:
                    if new_pci not in used_pcis:
                        df.loc[df['Cell name'] == cell_names[j], 'PCI'] = new_pci
                        used_pcis.add(new_pci)
                        break
    
    return pci_clashes

# Calculate distances and replan clashed PCIs in the "AGEGE" territory
pci_clashes = calculate_distances_and_replan_pci(df_agege)

# Display PCI clashes and the new DataFrame with updated PCI values
if pci_clashes:
    print("PCI Clashes found and replanned:")
    for clash in pci_clashes:
        print(f"Cells {clash[0]} and {clash[1]} had the same PCI ({clash[2]}) and were {clash[3]:.2f} km apart.")
    
    # Save the updated DataFrame to a CSV file
    df_agege.to_csv('updated_agege_cells.csv', index=False)
    print("\nUpdated DataFrame saved to 'updated_agege_cells.csv'.")
else:
    print("No PCI clashes found within the distance range of 0.1 km to 1 km.")


No PCI clashes found within the distance range of 0.1 km to 1 km.


### APAPA TERRITORY

In [17]:
# Filter DataFrame to only include cells in the "APAPA" territory
df_apapa = df[df['Territory'] == 'APAPA']

# Function to calculate distance between all cells within a specified territory and replan clashed PCIs
def calculate_distances_and_replan_pci(df, distance_range=(0.1, 1), available_pcis=range(0, 504)):
    pci_clashes = []
    cell_names = df['Cell name'].values
    latitudes = df['LAT'].values
    longitudes = df['LONG'].values
    pcis = df['PCI'].values
    used_pcis = set(pcis)
    
    for i in range(len(df)):
        coord1 = (latitudes[i], longitudes[i])
        
        for j in range(i + 1, len(df)):
            coord2 = (latitudes[j], longitudes[j])
            distance = geodesic(coord1, coord2).kilometers
            
            # Check for PCI clash within the specified distance range
            if distance_range[0] <= distance < distance_range[1] and pcis[i] == pcis[j]:
                pci_clashes.append((cell_names[i], cell_names[j], pcis[i], distance))
                
                # Find new PCI for one of the clashing cells
                for new_pci in available_pcis:
                    if new_pci not in used_pcis:
                        df.loc[df['Cell name'] == cell_names[j], 'PCI'] = new_pci
                        used_pcis.add(new_pci)
                        break
    
    return pci_clashes

# Calculate distances and replan clashed PCIs in the "APAPA" territory
pci_clashes = calculate_distances_and_replan_pci(df_apapa)

# Display PCI clashes and the new DataFrame with updated PCI values
if pci_clashes:
    print("PCI Clashes found and replanned:")
    for clash in pci_clashes:
        print(f"Cells {clash[0]} and {clash[1]} had the same PCI ({clash[2]}) and were {clash[3]:.2f} km apart.")
    
    # Save the updated DataFrame to a CSV file
    df_apapa.to_csv('updated_apapa_cells.csv', index=False)
    print("\nUpdated DataFrame saved to 'updated_apapa_cells.csv'.")
else:
    print("No PCI clashes found within the distance range of 0.1 km to 1 km.")


No PCI clashes found within the distance range of 0.1 km to 1 km.


### SURULERE, LAGOS, BADAGRY, EPE, OGUN WATERSIDE, ONDO, AND OGBOMOSHO TERRITORIES

In [18]:
# Filter DataFrame to only include cells in the specified territories
combined_territories = ['SURULERE', 'LAGOS', 'BADAGRY', 'EPE', 'OGUN WATERSIDE', 'ONDO', 'OGBOMOSHO']
df_combined = df[df['Territory'].isin(combined_territories)]

# Function to calculate distance between all cells within specified territories and replan clashed PCIs
def calculate_distances_and_replan_pci(df, distance_range=(0.1, 1), available_pcis=range(0, 504)):
    pci_clashes = []
    cell_names = df['Cell name'].values
    latitudes = df['LAT'].values
    longitudes = df['LONG'].values
    pcis = df['PCI'].values
    used_pcis = set(pcis)
    
    for i in range(len(df)):
        coord1 = (latitudes[i], longitudes[i])
        
        for j in range(i + 1, len(df)):
            coord2 = (latitudes[j], longitudes[j])
            distance = geodesic(coord1, coord2).kilometers
            
            # Check for PCI clash within the specified distance range
            if distance_range[0] <= distance < distance_range[1] and pcis[i] == pcis[j]:
                pci_clashes.append((cell_names[i], cell_names[j], pcis[i], distance))
                
                # Find new PCI for one of the clashing cells
                for new_pci in available_pcis:
                    if new_pci not in used_pcis:
                        df.loc[df['Cell name'] == cell_names[j], 'PCI'] = new_pci
                        used_pcis.add(new_pci)
                        break
    
    return pci_clashes

# Calculate distances and replan clashed PCIs in the combined territories
pci_clashes = calculate_distances_and_replan_pci(df_combined)

# Display PCI clashes and the new DataFrame with updated PCI values
if pci_clashes:
    print("PCI Clashes found and replanned:")
    for clash in pci_clashes:
        print(f"Cells {clash[0]} and {clash[1]} had the same PCI ({clash[2]}) and were {clash[3]:.2f} km apart.")
    
    # Save the updated DataFrame to a CSV file
    df_combined.to_csv('updated_combined_cells.csv', index=False)
    print("\nUpdated DataFrame saved to 'updated_combined_cells.csv'.")
else:
    print("No PCI clashes found within the distance range of 0.1 km to 1 km.")


No PCI clashes found within the distance range of 0.1 km to 1 km.


### LAGOS MAINLAND TERRITORY

In [19]:
# Filter DataFrame to only include cells in the "LAGOS MAINLAND" territory
df_lagos_mainland = df[df['Territory'] == 'LAGOS MAINLAND']

# Function to calculate distance between all cells within a specified territory and replan clashed PCIs
def calculate_distances_and_replan_pci(df, distance_range=(0.1, 1), available_pcis=range(0, 504)):
    pci_clashes = []
    cell_names = df['Cell name'].values
    latitudes = df['LAT'].values
    longitudes = df['LONG'].values
    pcis = df['PCI'].values
    used_pcis = set(pcis)
    
    for i in range(len(df)):
        coord1 = (latitudes[i], longitudes[i])
        
        for j in range(i + 1, len(df)):
            coord2 = (latitudes[j], longitudes[j])
            distance = geodesic(coord1, coord2).kilometers
            
            # Check for PCI clash within the specified distance range
            if distance_range[0] <= distance < distance_range[1] and pcis[i] == pcis[j]:
                pci_clashes.append((cell_names[i], cell_names[j], pcis[i], distance))
                
                # Find new PCI for one of the clashing cells
                for new_pci in available_pcis:
                    if new_pci not in used_pcis:
                        df.loc[df['Cell name'] == cell_names[j], 'PCI'] = new_pci
                        used_pcis.add(new_pci)
                        break
    
    return pci_clashes

# Calculate distances and replan clashed PCIs in the "LAGOS MAINLAND" territory
pci_clashes = calculate_distances_and_replan_pci(df_lagos_mainland)

# Display PCI clashes and the new DataFrame with updated PCI values
if pci_clashes:
    print("PCI Clashes found and replanned:")
    for clash in pci_clashes:
        print(f"Cells {clash[0]} and {clash[1]} had the same PCI ({clash[2]}) and were {clash[3]:.2f} km apart.")
    
    # Save the updated DataFrame to a CSV file
    df_lagos_mainland.to_csv('updated_lagos_mainland_cells.csv', index=False)
    print("\nUpdated DataFrame saved to 'updated_lagos_mainland_cells.csv'.")
else:
    print("No PCI clashes found within the distance range of 0.1 km to 1 km.")


PCI Clashes found and replanned:
Cells LLG2034A and LLG3982A had the same PCI (151) and were 0.95 km apart.
Cells LLG2034A and LLG3982D had the same PCI (151) and were 0.95 km apart.
Cells LLG2034A and LLG3982G had the same PCI (151) and were 0.95 km apart.
Cells LLG2034A and LLG3982J had the same PCI (151) and were 0.95 km apart.
Cells LLG2034A and LLG3982X had the same PCI (151) and were 0.95 km apart.
Cells LLG2034B and LLG3982B had the same PCI (150) and were 0.95 km apart.
Cells LLG2034B and LLG3982E had the same PCI (150) and were 0.95 km apart.
Cells LLG2034B and LLG3982H had the same PCI (150) and were 0.95 km apart.
Cells LLG2034B and LLG3982K had the same PCI (150) and were 0.95 km apart.
Cells LLG2034B and LLG3982Y had the same PCI (150) and were 0.95 km apart.
Cells LLG2034C and LLG3982C had the same PCI (152) and were 0.95 km apart.
Cells LLG2034C and LLG3982F had the same PCI (152) and were 0.95 km apart.
Cells LLG2034C and LLG3982I had the same PCI (152) and were 0.95 km

### IKORODU TERRITORY

In [20]:
# Filter DataFrame to only include cells in the "IKORODU" territory
df_ikorodu = df[df['Territory'] == 'IKORODU']

# Function to calculate distance between all cells within a specified territory and replan clashed PCIs
def calculate_distances_and_replan_pci(df, distance_range=(0.1, 1), available_pcis=range(0, 504)):
    pci_clashes = []
    cell_names = df['Cell name'].values
    latitudes = df['LAT'].values
    longitudes = df['LONG'].values
    pcis = df['PCI'].values
    used_pcis = set(pcis)
    
    for i in range(len(df)):
        coord1 = (latitudes[i], longitudes[i])
        
        for j in range(i + 1, len(df)):
            coord2 = (latitudes[j], longitudes[j])
            distance = geodesic(coord1, coord2).kilometers
            
            # Check for PCI clash within the specified distance range
            if distance_range[0] <= distance < distance_range[1] and pcis[i] == pcis[j]:
                pci_clashes.append((cell_names[i], cell_names[j], pcis[i], distance))
                
                # Find new PCI for one of the clashing cells
                for new_pci in available_pcis:
                    if new_pci not in used_pcis:
                        df.loc[df['Cell name'] == cell_names[j], 'PCI'] = new_pci
                        used_pcis.add(new_pci)
                        break
    
    return pci_clashes

# Calculate distances and replan clashed PCIs in the "IKORODU" territory
pci_clashes = calculate_distances_and_replan_pci(df_ikorodu)

# Display PCI clashes and the new DataFrame with updated PCI values
if pci_clashes:
    print("PCI Clashes found and replanned:")
    for clash in pci_clashes:
        print(f"Cells {clash[0]} and {clash[1]} had the same PCI ({clash[2]}) and were {clash[3]:.2f} km apart.")
    
    # Save the updated DataFrame to a CSV file
    df_ikorodu.to_csv('updated_ikorodu_cells.csv', index=False)
    print("\nUpdated DataFrame saved to 'updated_ikorodu_cells.csv'.")
else:
    print("No PCI clashes found within the distance range of 0.1 km to 1 km.")


No PCI clashes found within the distance range of 0.1 km to 1 km.


### MUSHIN TERRITORY

In [21]:
# Filter DataFrame to only include cells in the "MUSHIN" territory
df_mushin = df[df['Territory'] == 'MUSHIN']

# Function to calculate distance between all cells within a specified territory and replan clashed PCIs
def calculate_distances_and_replan_pci(df, distance_range=(0.1, 1), available_pcis=range(0, 504)):
    pci_clashes = []
    cell_names = df['Cell name'].values
    latitudes = df['LAT'].values
    longitudes = df['LONG'].values
    pcis = df['PCI'].values
    used_pcis = set(pcis)
    
    for i in range(len(df)):
        coord1 = (latitudes[i], longitudes[i])
        
        for j in range(i + 1, len(df)):
            coord2 = (latitudes[j], longitudes[j])
            distance = geodesic(coord1, coord2).kilometers
            
            # Check for PCI clash within the specified distance range
            if distance_range[0] <= distance < distance_range[1] and pcis[i] == pcis[j]:
                pci_clashes.append((cell_names[i], cell_names[j], pcis[i], distance))
                
                # Find new PCI for one of the clashing cells
                for new_pci in available_pcis:
                    if new_pci not in used_pcis:
                        df.loc[df['Cell name'] == cell_names[j], 'PCI'] = new_pci
                        used_pcis.add(new_pci)
                        break
    
    return pci_clashes

# Calculate distances and replan clashed PCIs in the "MUSHIN" territory
pci_clashes = calculate_distances_and_replan_pci(df_mushin)

# Display PCI clashes and the new DataFrame with updated PCI values
if pci_clashes:
    print("PCI Clashes found and replanned:")
    for clash in pci_clashes:
        print(f"Cells {clash[0]} and {clash[1]} had the same PCI ({clash[2]}) and were {clash[3]:.2f} km apart.")
    
    # Save the updated DataFrame to a CSV file
    df_mushin.to_csv('updated_mushin_cells.csv', index=False)
    print("\nUpdated DataFrame saved to 'updated_mushin_cells.csv'.")
else:
    print("No PCI clashes found within the distance range of 0.1 km to 1 km.")


No PCI clashes found within the distance range of 0.1 km to 1 km.


### SANGO TERRITORY

In [22]:
# Filter DataFrame to only include cells in the "SANGO" territory
df_sango = df[df['Territory'] == 'SANGO']

# Function to calculate distance between all cells within a specified territory and replan clashed PCIs
def calculate_distances_and_replan_pci(df, distance_range=(0.1, 1), available_pcis=range(0, 504)):
    pci_clashes = []
    cell_names = df['Cell name'].values
    latitudes = df['LAT'].values
    longitudes = df['LONG'].values
    pcis = df['PCI'].values
    used_pcis = set(pcis)
    
    for i in range(len(df)):
        coord1 = (latitudes[i], longitudes[i])
        
        for j in range(i + 1, len(df)):
            coord2 = (latitudes[j], longitudes[j])
            distance = geodesic(coord1, coord2).kilometers
            
            # Check for PCI clash within the specified distance range
            if distance_range[0] <= distance < distance_range[1] and pcis[i] == pcis[j]:
                pci_clashes.append((cell_names[i], cell_names[j], pcis[i], distance))
                
                # Find new PCI for one of the clashing cells
                for new_pci in available_pcis:
                    if new_pci not in used_pcis:
                        df.loc[df['Cell name'] == cell_names[j], 'PCI'] = new_pci
                        used_pcis.add(new_pci)
                        break
    
    return pci_clashes

# Calculate distances and replan clashed PCIs in the "SANGO" territory
pci_clashes = calculate_distances_and_replan_pci(df_sango)

# Display PCI clashes and the new DataFrame with updated PCI values
if pci_clashes:
    print("PCI Clashes found and replanned:")
    for clash in pci_clashes:
        print(f"Cells {clash[0]} and {clash[1]} had the same PCI ({clash[2]}) and were {clash[3]:.2f} km apart.")
    
    # Save the updated DataFrame to a CSV file
    df_sango.to_csv('updated_sango_cells.csv', index=False)
    print("\nUpdated DataFrame saved to 'updated_sango_cells.csv'.")
else:
    print("No PCI clashes found within the distance range of 0.1 km to 1 km.")


No PCI clashes found within the distance range of 0.1 km to 1 km.


### FESTAC TERRITORY

In [23]:
# Filter DataFrame to only include cells in the "FESTAC" territory
df_festac = df[df['Territory'] == 'FESTAC']

# Function to calculate distance between all cells within a specified territory and replan clashed PCIs
def calculate_distances_and_replan_pci(df, distance_range=(0.1, 1), available_pcis=range(0, 504)):
    pci_clashes = []
    cell_names = df['Cell name'].values
    latitudes = df['LAT'].values
    longitudes = df['LONG'].values
    pcis = df['PCI'].values
    used_pcis = set(pcis)
    
    for i in range(len(df)):
        coord1 = (latitudes[i], longitudes[i])
        
        for j in range(i + 1, len(df)):
            coord2 = (latitudes[j], longitudes[j])
            distance = geodesic(coord1, coord2).kilometers
            
            # Check for PCI clash within the specified distance range
            if distance_range[0] <= distance < distance_range[1] and pcis[i] == pcis[j]:
                pci_clashes.append((cell_names[i], cell_names[j], pcis[i], distance))
                
                # Find new PCI for one of the clashing cells
                for new_pci in available_pcis:
                    if new_pci not in used_pcis:
                        df.loc[df['Cell name'] == cell_names[j], 'PCI'] = new_pci
                        used_pcis.add(new_pci)
                        break
    
    return pci_clashes

# Calculate distances and replan clashed PCIs in the "FESTAC" territory
pci_clashes = calculate_distances_and_replan_pci(df_festac)

# Display PCI clashes and the new DataFrame with updated PCI values
if pci_clashes:
    print("PCI Clashes found and replanned:")
    for clash in pci_clashes:
        print(f"Cells {clash[0]} and {clash[1]} had the same PCI ({clash[2]}) and were {clash[3]:.2f} km apart.")
    
    # Save the updated DataFrame to a CSV file
    df_festac.to_csv('updated_festac_cells.csv', index=False)
    print("\nUpdated DataFrame saved to 'updated_festac_cells.csv'.")
else:
    print("No PCI clashes found within the distance range of 0.1 km to 1 km.")


PCI Clashes found and replanned:
Cells LLG1057P and LLG2658B had the same PCI (208) and were 0.76 km apart.
Cells LLG1057P and LLG2658E had the same PCI (208) and were 0.76 km apart.
Cells LLG1057P and LLG2658H had the same PCI (208) and were 0.76 km apart.

Updated DataFrame saved to 'updated_festac_cells.csv'.


### ABEOKUTA TERRITORY

In [24]:
# Filter DataFrame to only include cells in the "ABEOKUTA" territory
df_abeokuta = df[df['Territory'] == 'ABEOKUTA']

# Function to calculate distance between all cells within a specified territory and replan clashed PCIs
def calculate_distances_and_replan_pci(df, distance_range=(0.1, 1), available_pcis=range(0, 504)):
    pci_clashes = []
    cell_names = df['Cell name'].values
    latitudes = df['LAT'].values
    longitudes = df['LONG'].values
    pcis = df['PCI'].values
    used_pcis = set(pcis)
    
    for i in range(len(df)):
        coord1 = (latitudes[i], longitudes[i])
        
        for j in range(i + 1, len(df)):
            coord2 = (latitudes[j], longitudes[j])
            distance = geodesic(coord1, coord2).kilometers
            
            # Check for PCI clash within the specified distance range
            if distance_range[0] <= distance < distance_range[1] and pcis[i] == pcis[j]:
                pci_clashes.append((cell_names[i], cell_names[j], pcis[i], distance))
                
                # Find new PCI for one of the clashing cells
                for new_pci in available_pcis:
                    if new_pci not in used_pcis:
                        df.loc[df['Cell name'] == cell_names[j], 'PCI'] = new_pci
                        used_pcis.add(new_pci)
                        break
    
    return pci_clashes

# Calculate distances and replan clashed PCIs in the "ABEOKUTA" territory
pci_clashes = calculate_distances_and_replan_pci(df_abeokuta)

# Display PCI clashes and the new DataFrame with updated PCI values
if pci_clashes:
    print("PCI Clashes found and replanned:")
    for clash in pci_clashes:
        print(f"Cells {clash[0]} and {clash[1]} had the same PCI ({clash[2]}) and were {clash[3]:.2f} km apart.")
    
    # Save the updated DataFrame to a CSV file
    df_abeokuta.to_csv('updated_abeokuta_cells.csv', index=False)
    print("\nUpdated DataFrame saved to 'updated_abeokuta_cells.csv'.")
else:
    print("No PCI clashes found within the distance range of 0.1 km to 1 km.")


No PCI clashes found within the distance range of 0.1 km to 1 km.


### ALIMOSHO TERRITORY

In [25]:
# Filter DataFrame to only include cells in the "ALIMOSHO" territory
df_alimosho = df[df['Territory'] == 'ALIMOSHO']

# Function to calculate distance between all cells within a specified territory and replan clashed PCIs
def calculate_distances_and_replan_pci(df, distance_range=(0.1, 1), available_pcis=range(0, 504)):
    pci_clashes = []
    cell_names = df['Cell name'].values
    latitudes = df['LAT'].values
    longitudes = df['LONG'].values
    pcis = df['PCI'].values
    used_pcis = set(pcis)
    
    for i in range(len(df)):
        coord1 = (latitudes[i], longitudes[i])
        
        for j in range(i + 1, len(df)):
            coord2 = (latitudes[j], longitudes[j])
            distance = geodesic(coord1, coord2).kilometers
            
            # Check for PCI clash within the specified distance range
            if distance_range[0] <= distance < distance_range[1] and pcis[i] == pcis[j]:
                pci_clashes.append((cell_names[i], cell_names[j], pcis[i], distance))
                
                # Find new PCI for one of the clashing cells
                for new_pci in available_pcis:
                    if new_pci not in used_pcis:
                        df.loc[df['Cell name'] == cell_names[j], 'PCI'] = new_pci
                        used_pcis.add(new_pci)
                        break
    
    return pci_clashes

# Calculate distances and replan clashed PCIs in the "ALIMOSHO" territory
pci_clashes = calculate_distances_and_replan_pci(df_alimosho)

# Display PCI clashes and the new DataFrame with updated PCI values
if pci_clashes:
    print("PCI Clashes found and replanned:")
    for clash in pci_clashes:
        print(f"Cells {clash[0]} and {clash[1]} had the same PCI ({clash[2]}) and were {clash[3]:.2f} km apart.")
    
    # Save the updated DataFrame to a CSV file
    df_alimosho.to_csv('updated_alimosho_cells.csv', index=False)
    print("\nUpdated DataFrame saved to 'updated_alimosho_cells.csv'.")
else:
    print("No PCI clashes found within the distance range of 0.1 km to 1 km.")


PCI Clashes found and replanned:
Cells L6265A and LLG5788A had the same PCI (387) and were 0.95 km apart.
Cells L6265A and LLG5788D had the same PCI (387) and were 0.95 km apart.
Cells L6265A and LLG5788G had the same PCI (387) and were 0.95 km apart.
Cells L6265A and LLG5788X had the same PCI (387) and were 0.95 km apart.
Cells L6265B and LLG5788B had the same PCI (388) and were 0.95 km apart.
Cells L6265B and LLG5788E had the same PCI (388) and were 0.95 km apart.
Cells L6265B and LLG5788H had the same PCI (388) and were 0.95 km apart.
Cells L6265B and LLG5788Y had the same PCI (388) and were 0.95 km apart.
Cells L6265C and LLG5788C had the same PCI (389) and were 0.95 km apart.
Cells L6265C and LLG5788F had the same PCI (389) and were 0.95 km apart.
Cells L6265C and LLG5788I had the same PCI (389) and were 0.95 km apart.
Cells L6265C and LLG5788Z had the same PCI (389) and were 0.95 km apart.

Updated DataFrame saved to 'updated_alimosho_cells.csv'.


### IJEBU-ODE TERRITORY

In [26]:
# Filter DataFrame to only include cells in the "IJEBU-ODE" territory
df_ijebu_ode = df[df['Territory'] == 'IJEBU-ODE']

# Function to calculate distance between all cells within a specified territory and replan clashed PCIs
def calculate_distances_and_replan_pci(df, distance_range=(0.1, 1), available_pcis=range(0, 504)):
    pci_clashes = []
    cell_names = df['Cell name'].values
    latitudes = df['LAT'].values
    longitudes = df['LONG'].values
    pcis = df['PCI'].values
    used_pcis = set(pcis)
    
    for i in range(len(df)):
        coord1 = (latitudes[i], longitudes[i])
        
        for j in range(i + 1, len(df)):
            coord2 = (latitudes[j], longitudes[j])
            distance = geodesic(coord1, coord2).kilometers
            
            # Check for PCI clash within the specified distance range
            if distance_range[0] <= distance < distance_range[1] and pcis[i] == pcis[j]:
                pci_clashes.append((cell_names[i], cell_names[j], pcis[i], distance))
                
                # Find new PCI for one of the clashing cells
                for new_pci in available_pcis:
                    if new_pci not in used_pcis:
                        df.loc[df['Cell name'] == cell_names[j], 'PCI'] = new_pci
                        used_pcis.add(new_pci)
                        break
    
    return pci_clashes

# Calculate distances and replan clashed PCIs in the "IJEBU-ODE" territory
pci_clashes = calculate_distances_and_replan_pci(df_ijebu_ode)

# Display PCI clashes and the new DataFrame with updated PCI values
if pci_clashes:
    print("PCI Clashes found and replanned:")
    for clash in pci_clashes:
        print(f"Cells {clash[0]} and {clash[1]} had the same PCI ({clash[2]}) and were {clash[3]:.2f} km apart.")
    
    # Save the updated DataFrame to a CSV file
    df_ijebu_ode.to_csv('updated_ijebu_ode_cells.csv', index=False)
    print("\nUpdated DataFrame saved to 'updated_ijebu_ode_cells.csv'.")
else:
    print("No PCI clashes found within the distance range of 0.1 km to 1 km.")


No PCI clashes found within the distance range of 0.1 km to 1 km.


### LAGOS ISLAND TERRITORY

In [27]:
# Filter DataFrame to only include cells in the "LAGOS ISLAND" territory
df_lagos_island = df[df['Territory'] == 'LAGOS ISLAND']

# Function to calculate distance between all cells within a specified territory and replan clashed PCIs
def calculate_distances_and_replan_pci(df, distance_range=(0.1, 1), available_pcis=range(0, 504)):
    pci_clashes = []
    cell_names = df['Cell name'].values
    latitudes = df['LAT'].values
    longitudes = df['LONG'].values
    pcis = df['PCI'].values
    used_pcis = set(pcis)
    
    for i in range(len(df)):
        coord1 = (latitudes[i], longitudes[i])
        
        for j in range(i + 1, len(df)):
            coord2 = (latitudes[j], longitudes[j])
            distance = geodesic(coord1, coord2).kilometers
            
            # Check for PCI clash within the specified distance range
            if distance_range[0] <= distance < distance_range[1] and pcis[i] == pcis[j]:
                pci_clashes.append((cell_names[i], cell_names[j], pcis[i], distance))
                
                # Find new PCI for one of the clashing cells
                for new_pci in available_pcis:
                    if new_pci not in used_pcis:
                        df.loc[df['Cell name'] == cell_names[j], 'PCI'] = new_pci
                        used_pcis.add(new_pci)
                        break
    
    return pci_clashes

# Calculate distances and replan clashed PCIs in the "LAGOS ISLAND" territory
pci_clashes = calculate_distances_and_replan_pci(df_lagos_island)

# Display PCI clashes and the new DataFrame with updated PCI values
if pci_clashes:
    print("PCI Clashes found and replanned:")
    for clash in pci_clashes:
        print(f"Cells {clash[0]} and {clash[1]} had the same PCI ({clash[2]}) and were {clash[3]:.2f} km apart.")
    
    # Save the updated DataFrame to a CSV file
    df_lagos_island.to_csv('updated_lagos_island_cells.csv', index=False)
    print("\nUpdated DataFrame saved to 'updated_lagos_island_cells.csv'.")
else:
    print("No PCI clashes found within the distance range of 0.1 km to 1 km.")


PCI Clashes found and replanned:
Cells L6400D and LLG1996C had the same PCI (35) and were 0.79 km apart.
Cells L6400D and LLG1996F had the same PCI (35) and were 0.79 km apart.
Cells L6400D and LLG1996I had the same PCI (35) and were 0.79 km apart.
Cells L6400D and LLG1996L had the same PCI (35) and were 0.79 km apart.
Cells L6400D and LLG1996Z had the same PCI (35) and were 0.79 km apart.
Cells L6400D and LLG5991C had the same PCI (35) and were 0.66 km apart.
Cells L6400D and LLG5991F had the same PCI (35) and were 0.66 km apart.
Cells L6400D and LLG5991I had the same PCI (35) and were 0.66 km apart.
Cells L6400D and LLG5991L had the same PCI (35) and were 0.66 km apart.
Cells L6400E and LLG5991A had the same PCI (33) and were 0.66 km apart.
Cells L6400E and LLG5991D had the same PCI (33) and were 0.66 km apart.
Cells L6400E and LLG5991G had the same PCI (33) and were 0.66 km apart.
Cells L6400E and LLG5991J had the same PCI (33) and were 0.66 km apart.
Cells L6400F and LLG5991B had t